In [1]:
import pandas as pd
import datetime
import config

In [2]:
path_temp = config.PATH_TEMP_DATA
path_visuals = config.PATH_VIZ
path_source_data = config.PATH_SOURCE_DATA
path_output_data = config.PATH_OUTPUT_DATA

In [3]:
# get a all elections

elections = pd.read_csv(path_source_data + 'view_election.csv')
len(elections)

8146

In [4]:
# remove ep elections

elections = elections[elections['election_type'] == 'parliament']
len(elections)

6736

In [5]:
# extract election year

elections['election_year'] = pd.to_datetime(elections['election_date']).dt.year

In [6]:
# select necessary columns

elections = elections[['country_name', 
                       'country_name_short', 
                       'vote_share', 
                       'seats', 
                       'party_id', 
                       'party_name', 
                       'party_name_english',
                       'election_year']]

In [10]:
# merge with party info to get party family names

party = pd.read_csv(path_source_data + 'view_party.csv')
party = party[['party_id', 'family_id', 'family_name', 'left_right', 'liberty_authority', 'eu_anti_pro']]
df = pd.merge(elections, party, on='party_id', how='left')
df = df.drop_duplicates()
len(df)

6733

In [11]:
# remove non_eu members

non_eu_members = ['Norway', 'Canada', 'Australia', 'Switzerland', 'New Zealand', 'Iceland', 'Japan',
                 'Israel', 'Turkey']

df = df[~df['country_name'].isin(non_eu_members)]
len(df)

4725

In [12]:
# reshape data so that every year has values, not only election years

df2 = df.pivot_table(index=['election_year'], 
                     columns=['country_name',
                              'party_id', 
                              'family_name'], 
                     values=['seats', 
                             'vote_share', 
                             'left_right', 
                             'liberty_authority', 
                             'eu_anti_pro']).stack([1,2,3])


In [13]:
# fill the NaNs

df2 = df2.apply(lambda series: series.loc[:series.last_valid_index()].ffill())

In [21]:
# try to find changes in number of parties per country

df3 = df2.groupby(['election_year', 'country_name', 'party_id'])['vote_share'].mean()

In [23]:
df3.to_csv(path_visuals + 'vis_v3.csv', header=True)

In [28]:
df4 = df3.groupby(['election_year', 'country_name']).count()

In [29]:
df4.to_csv(path_visuals + 'vis_v4.csv', header=True)